In [1]:
from song_scraper import song_scraper
import pandas as pd

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)
pd.set_option("max_colwidth", 800)
pd.set_option("display.max_colwidth", None)

In [2]:
artists = ["Eminem"]

artists = [x.replace(" ", "_").lower() for x in artists]
artists

['eminem']

In [3]:
df = song_scraper(artists)

c:\Users\wilko\Desktop\Studia\3 rok\Projekty studia\Python-Projekt\song_scraper.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  song_page = requests.get(row[0])


In [4]:
df.head()

Polish  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [18]:
df.duplicated().sum()

0